# Softmax回帰の実装
- irisに対し,softmax回帰で実装する

In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()

In [22]:
import torch
from torch import nn
import torch.utils.data

In [29]:
def train_epoch(model, data_loader):
    model.eval()
    for batch_idx, (data, target) in enumerate(data_loader): # 入力と正解
         optimizer.zero_grad() # Weightの初期化
         output = model(data) # 仮説で値代入
         output.dtype
         loss = criterion(output, target) # 損失
         loss.backward() # 微分の計算
         optimizer.step() # パラメータの更新
         if batch_idx % 10 == 0:
             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                 batch_idx, batch_idx * len(data), len(data_loader.dataset),
                 100. * batch_idx / len(data_loader), loss.item()))

def valid_epoch(model, data_loader):
    model.train()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader): # 入力と正解
             optimizer.zero_grad() # Weightの初期化
             output = model(data) # 仮説で値代入
             output.dtype
             loss = criterion(output, target) # 損失
             # 本来は全体でロスを数えて荷重平均を取る,accuracyを計算する

             if batch_idx % 10 == 0:
                 print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                     batch_idx, batch_idx * len(data), len(data_loader.dataset),
                     100. * batch_idx / len(data_loader), loss.item()))

In [30]:
iris = load_iris()
X_train, X_valid, y_train, y_valid = train_test_split(iris.data, iris.target, test_size=0.2)

X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train)
X_valid = torch.tensor(X_valid).float()
y_valid = torch.tensor(y_valid)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
valid_dataset = torch.utils.data.TensorDataset(X_valid, y_valid)

model = nn.Linear(4, 3)

batch_size  = 120 # ミニバッチのデータの数
max_epoch = 100 #
train_loader = torch.utils.data.DataLoader(train_dataset, 
                   batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(train_dataset, 
                   batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss() # 損失の定義
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) #(確率的)勾配降下法

In [31]:
for epoch in range(max_epoch):
    train_epoch(model, train_loader)
    valid_epoch(model, valid_loader)

Train Epoch: 0 [0/120 (0%)]	Loss: 2.696589
Test Epoch: 0 [0/120 (0%)]	Loss: 2.541242
Train Epoch: 0 [0/120 (0%)]	Loss: 2.541242
Test Epoch: 0 [0/120 (0%)]	Loss: 2.408026
Train Epoch: 0 [0/120 (0%)]	Loss: 2.408026
Test Epoch: 0 [0/120 (0%)]	Loss: 2.289122
Train Epoch: 0 [0/120 (0%)]	Loss: 2.289121
Test Epoch: 0 [0/120 (0%)]	Loss: 2.179420
Train Epoch: 0 [0/120 (0%)]	Loss: 2.179420
Test Epoch: 0 [0/120 (0%)]	Loss: 2.076118
Train Epoch: 0 [0/120 (0%)]	Loss: 2.076119
Test Epoch: 0 [0/120 (0%)]	Loss: 1.977985
Train Epoch: 0 [0/120 (0%)]	Loss: 1.977985
Test Epoch: 0 [0/120 (0%)]	Loss: 1.884750
Train Epoch: 0 [0/120 (0%)]	Loss: 1.884750
Test Epoch: 0 [0/120 (0%)]	Loss: 1.796715
Train Epoch: 0 [0/120 (0%)]	Loss: 1.796714
Test Epoch: 0 [0/120 (0%)]	Loss: 1.714491
Train Epoch: 0 [0/120 (0%)]	Loss: 1.714491
Test Epoch: 0 [0/120 (0%)]	Loss: 1.638829
Train Epoch: 0 [0/120 (0%)]	Loss: 1.638828
Test Epoch: 0 [0/120 (0%)]	Loss: 1.570448
Train Epoch: 0 [0/120 (0%)]	Loss: 1.570448
Test Epoch: 0 [0/120 (

In [13]:
y_valid

array([1, 2, 1, 2, 2, 2, 0, 1, 1, 1, 0, 0, 0, 0, 2, 1, 1, 1, 2, 1, 1, 0,
       1, 2, 2, 1, 2, 1, 2, 1])

In [17]:
X_valid.shape

(30, 4)

In [18]:
X_train.shape

torch.Size([120, 4])